In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import colorcet as cc

from pathlib import Path
from scipy import signal, optimize
from spt3g import mapmaker
from spt3g.core import G3File
from spt3g.pointing import focus

from focus_tools import * 

In [ ]:
source = 'PMNJ0210-5101-pixelraster'
bolodata_dir = Path('/spt_data/bolodata/fullrate') / source
map_dir = Path('/poleanalysis/gnoble/calresult/calibration') / source

# glob all the observations of interest
obs_paths = sorted(map_dir.glob('*.g3'))
obs_ids = [str(obs).split('/')[-1].split('.')[0] for obs in obs_paths]
print('found {} observations'.format(len(obs_paths)))

In [ ]:
# some analysis choices
annulus_inner, annulus_outer = 4.0, 10.0 # arcmin
crop_shape = np.array([20., 20.]) # size to crop the map to, in arcmin
zoom_shape = np.array([8., 8.]) # size to zoom in on, after fitting, in arcmin

In [ ]:
# define a fit model, a 2D Gaussian in this case
def fit_model(yx, amplitude, y0, x0, fwhm, ellip, theta, offset, flat=True):
    y, x = yx
    var_y, var_x = fwhm2var(fwhm, ellip)
    res = gaussian_2d(y, x, y0, x0, var_y, var_x, theta, norm=amplitude) + offset
    if flat:
        return res.flat
    return res

# and the initial parameters, set by hand for a little added robustness
# [amp, y0, x0, fwhm, ellip, theta, offset] in pixel units
initial_params = {'90GHz': [200., 49., 45., 7.5, 0.1, 0.5, 1.0],
                  '150GHz': [20., 49., 45., 6.5, 0.1, 0.5, 1.0],
                  '220GHz': [10., 49., 45., 6.0, 0.1, 0.5, 1.0]}

In [ ]:
# initialize a results dict, to be keyed by obs_id , wafer, and band
results = {obs:{} for obs in obs_ids}

for i, (obs_path, obs_id) in enumerate(zip(obs_paths, obs_ids)):

    # retrieve the bench and optics positions for this observation first
    bolodata_path = bolodata_dir / obs_id / '0000.g3'
    try:
        for frame in G3File(str(bolodata_path)):
            try:
                # XYZ bench coordinates
                results[obs_id]['bench'] = focus.g3bench2xyz(frame['BenchCommandedPosition'])
            except KeyError:
                continue
            else:
                # coordinates relative to the optical axis
                results[obs_id]['optic'] = np.array(focus.bench2optical(*results[obs_id]['bench'])).flatten()[::-1]
                break
        else:
            print('BenchCommandedPosition not found in {}!'.format(bolodata_path))
    except RuntimeError as err: # can't find bolodata, probably
        print('RuntimeError encountered in trying to retrieve focus position from {}:\n{}'.format(bolodata_path, err))

    # now loop over frames (maps) in the g3 file    
    for j, frame in enumerate(G3File(str(obs_path))):
        band, wafer = frame['Id'].split('-')[-2:]
      
        try:
            results[obs_id][wafer][band] = {}
        except KeyError: # haven't seen this wafer yet
            results[obs_id][wafer] = {band: {}}
        
        # fetch the temperature map, unweight, and convert to mK
        t_map = 1000.*frame['T']/frame['Wunpol'].TT
        res_arcmin = rad2arcmin(t_map.res)
        
        # center-crop to a more reasonable size
        t_map = crop(np.array(t_map), crop_shape/res_arcmin)
        
        # median filter so we can estimate source location
        t_map_filtered = signal.medfilt2d(t_map, kernel_size=11)
        
        try: # pick the median-filtered max and argmax
            t_max = t_map_filtered[np.isfinite(t_map_filtered)].max()
            t_argmax = np.unravel_index(np.argmax(t_map_filtered), t_map_filtered.shape)
        except ValueError as err: # we have no good pixels
            print('{} | {} | {} | {}'.format(obs_id, wafer, band, err))
            results[obs_id][wafer][band] = {'t_map': t_map,
                                            'res_arcmin': res_arcmin,
                                            't_argmax': (np.nan, np.nan),
                                            't_map_annulus': np.nan,
                                            'annular_mean': np.nan,
                                            'annular_std': np.nan,
                                            'initial_params': np.nan*np.empty(7, float),
                                            'fit_params': np.nan*np.empty(7, dtype=float),
                                            'fit_cov': np.nan*np.empty((7, 7), dtype=float)}
            continue
    
        # subset annular region and compute its mean and variance
        t_map_annulus = t_map[annular_indices(t_map.shape, annulus_inner/res_arcmin, 
                                              annulus_outer/res_arcmin, *t_argmax)]
        t_map_annulus = t_map_annulus[np.all([t_map_annulus != 0., np.isfinite(t_map_annulus)], axis=0)]
        annular_mean, annular_std = t_map_annulus.mean(), t_map_annulus.std()

        # store these results now, before the fit
        results[obs_id][wafer][band] = {'t_map': t_map,
                                        'res_arcmin': res_arcmin,
                                        't_argmax': t_argmax,
                                        't_map_annulus': t_map_annulus,
                                        'annular_mean': annular_mean,
                                        'annular_std': annular_std}
        
        # least-squares fit with `curve_fit` for maximum black-box
        yx = np.indices(t_map.shape)
        good_inds = np.all([t_map != 0., np.isfinite(t_map)], axis=0)
        try:
            fit_params, fit_cov = optimize.curve_fit(fit_model, yx[:, good_inds], t_map[good_inds].flat,
                                                     p0=initial_params[band], maxfev=1000,
                                                     sigma=np.repeat(annular_std, np.sum(good_inds)),
                                                     absolute_sigma=True)
        except Exception as err:
            print('{} | {} | {} | {}'.format(obs_id, wafer, band, err))
            results[obs_id][wafer][band]['fit_params'] = np.nan*np.empty(7, dtype=float)
            results[obs_id][wafer][band]['fit_cov'] = np.nan*np.empty((7, 7), dtype=float)
        else:
            results[obs_id][wafer][band]['fit_params'] = fit_params
            results[obs_id][wafer][band]['fit_cov'] = fit_cov

        # record some fit parameters explicitly
        amp = results[obs_id][wafer][band]['fit_params'][0]
        amp_err = np.sqrt(results[obs_id][wafer][band]['fit_cov'][0, 0])
        
        snr = amp/results[obs_id][wafer][band]['annular_std']
        
        fwhm, ellip, theta = results[obs_id][wafer][band]['fit_params'][3:6]
        fwhm_err, ellip_err, theta_err = np.sqrt(np.diag(results[obs_id][wafer][band]['fit_cov'])[3:6])
        
        fwhm *= results[obs_id][wafer][band]['res_arcmin']
        fwhm_err *= results[obs_id][wafer][band]['res_arcmin']
                                            
        results[obs_id][wafer][band].update({'snr': snr, 'amp':amp, 'amp_err': amp_err, 'fwhm': fwhm,
                                             'fwhm_err': fwhm_err, 'ellip': ellip, 'ellip_err': ellip_err,
                                             'theta': theta, 'theta_err': theta_err})


In [ ]:
# plotting constants for consistency
mpl.rcParams['font.size'] = 16
imshow_kwargs = {'interpolation':'none', 'aspect':'equal', 'cmap':cc.cm['fire'], 'vmin':-0.1, 'vmax':0.9, 'origin':'lower'}
hist_kwargs = {'bins':40, 'facecolor':'dimgrey', 'edgecolor':'none'}
patch_kwargs = {'edgecolor':'white', 'facecolor':'none', 'linewidth':2., 'linestyle':'dotted'}
line_kwargs = {'linewidth':2., 'color':'firebrick'}
contour_kwargs = {'colors':'white', 'linewidths':[2., 2.], 'linestyles':['dotted', 'solid']}
annotate_kwargs = {'xycoords':'axes fraction', 'color':'white', 'horizontalalignment':'right'}

In [ ]:
bands = ['90GHz', '150GHz', '220GHz']
wafers = sorted([k for k in results[obs_id].keys() if k.startswith('w')])

In [ ]:
t_norms = np.empty(3, float)
for i, band in enumerate(bands):
    t_norms[i] = np.nanmedian([[val[wafer][band]['fit_params'][0] for val in results.values()]
                               for wafer in wafers])

In [ ]:
# first, plot t-maps with noise annuli overlaid
for b, band in enumerate(bands):
    fig, ax = plt.subplots(len(wafers), len(obs_paths), figsize=(4*len(obs_ids), 4*len(wafers)))
    
    # normalize the colormap by the average fitted amplitude
    t_norm = np.nanmedian([val[wafer][band]['fit_params'][0] for val in results.values()])
    
    for i, wafer in enumerate(wafers):
        
        ax[i, 0].set_ylabel('{}, {}'.format(wafer, band), fontsize=20)
        
        for j, obs_id in enumerate(obs_ids):
            title = 'Obs{}\n({:+05.1f}, {:+05.1f}, {:+05.1f})\n({:+05.1f}, {:+05.1f}, {:+05.1f})'.format(
                obs_id, *results[obs_id]['bench'], *results[obs_id]['optic'].flat)
            
            ax[0, j].set_title(title)

            ax[i, j].imshow(results[obs_id][wafer][band]['t_map']/t_norms[b], **imshow_kwargs)

            ax[i, j].add_patch(mpl.patches.Circle(results[obs_id][wafer][band]['t_argmax'][::-1],
                                                  annulus_inner/results[obs_id][wafer][band]['res_arcmin'],
                                                  **patch_kwargs))
            ax[i, j].add_patch(mpl.patches.Circle(results[obs_id][wafer][band]['t_argmax'][::-1],
                                                  annulus_outer/results[obs_id][wafer][band]['res_arcmin'],
                                                  **patch_kwargs))

            ax[i, j].set_xticks([]); ax[i, j].set_yticks([])

    fig.tight_layout()

In [ ]:
# histogram the noise annuli
for b, band in enumerate(bands):
    fig, ax = plt.subplots(len(wafers), len(obs_paths), figsize=(4*len(obs_ids), 4*len(wafers)), sharey='row')

    for i, wafer in enumerate(wafers):
        
        ax[i, 0].set_ylabel(wafer, fontsize=20)
        ax[i, 0].set_yticks([])
        
        for j, obs_id in enumerate(obs_ids):
            title = 'Obs{}\n({:+05.1f}, {:+05.1f}, {:+05.1f})\n({:+05.1f}, {:+05.1f}, {:+05.1f})'.format(
                obs_id, *results[obs_id]['bench'], *results[obs_id]['optic'].flat)
            
            ax[0, j].set_title(title)

            # range to 3-'sigma'
            mean, std = results[obs_id][wafer][band]['annular_mean'], results[obs_id][wafer][band]['annular_std']
            hist_range = (mean - 3.*std, mean + 3.*std)

            try:
                ax[i, j].hist(results[obs_id][wafer][band]['t_map_annulus'], range=hist_range, normed=True,
                              **hist_kwargs)
            except ValueError:
                pass
            else:
                # and overplot a moment-matched Gaussian
                x = np.linspace(*hist_range, 100)
                ax[i, j].plot(x, gaussian_1d(x, mean, std*std), **line_kwargs)
            
            ax[i, j].annotate(r'$\sigma={:1.2f}\,$mK'.format(std),
                                   (0.95, 0.85), xycoords='axes fraction', horizontalalignment='right')

            ax[-1, j].set_xlabel('$\Delta$T [mK$_\mathrm{CMB}$]')

    fig.tight_layout()

In [ ]:
# zoom in on t-maps with fitted Gaussian contours
for b, band in enumerate(bands):
    fig, ax = plt.subplots(len(wafers), len(obs_paths), figsize=(4*len(obs_ids), 4*len(wafers)), sharey='row')

    for i, wafer in enumerate(wafers):
        
        ax[i, 0].set_ylabel(wafer, fontsize=20)
        ax[i, 0].set_yticks([])
        
        for j, obs_id in enumerate(obs_ids):
            title = 'Obs{}\n({:+05.1f}, {:+05.1f}, {:+05.1f})\n({:+05.1f}, {:+05.1f}, {:+05.1f})'.format(
                obs_id, *results[obs_id]['bench'], *results[obs_id]['optic'].flat)
            
            ax[0, j].set_title(title)

#             t_map_zoom = crop(results[obs_id][wafer][band]['t_map'], zoom_shape/results[obs_id][wafer][band]['res_arcmin'],
#                               results[obs_id][wafer][band]['fit_params'][1], results[obs_id][wafer][band]['fit_params'][2])

            t_map_zoom = crop(results[obs_id][wafer][band]['t_map'], zoom_shape/results[obs_id][wafer][band]['res_arcmin'])
            
            ax[i ,j].imshow(t_map_zoom/t_norms[b], **imshow_kwargs)
            
            yx_zoom = np.indices(t_map_zoom.shape)

#             t_map_fit = crop(fit_model(yx, *results[obs_id][wafer][band]['fit_params'], flat=False),
#                              np.array(zoom_shape)/results[obs_id][wafer][band]['res_arcmin'],
#                              results[obs_id][wafer][band]['fit_params'][1], results[obs_id][wafer][band]['fit_params'][2])
            
            t_map_fit = crop(fit_model(yx, *results[obs_id][wafer][band]['fit_params'], flat=False),
                             np.array(zoom_shape)/results[obs_id][wafer][band]['res_arcmin'])

            ax[i, j].contour(yx_zoom[1], yx_zoom[0], t_map_fit-results[obs_id][wafer][band]['fit_params'][6],
                                 [results[obs_id][wafer][band]['fit_params'][0]*0.05399,
                                  results[obs_id][wafer][band]['fit_params'][0]*0.5], **contour_kwargs)

            ax[i, j].annotate(r"S/N $\simeq$ {snr:1.0f}".format(**results[obs_id][wafer][band]),
                                 (0.95, 0.9), **annotate_kwargs)
            ax[i, j].annotate(r"FWHM = {fwhm:1.3f} $\pm$ {fwhm_err:1.3f}'".format(**results[obs_id][wafer][band]),
                                 (0.95, 0.15), **annotate_kwargs)
            ax[i, j].annotate(r"|e| = {ellip:1.3f} $\pm$ {ellip_err:1.3f}".format(**results[obs_id][wafer][band]),
                                 (0.95, 0.05), **annotate_kwargs)

            ax[i, j].set_xticks([]); ax[i, j].set_yticks([])

    fig.tight_layout()

In [ ]:
# make a table of fit parameters
row_str = '\t{obs}/{wafer}\t{amp:05.1f} +/- {amp_err:02.1f}\t\t{fwhm:04.2f}  +/- {fwhm_err:04.2f}\t\t' \
          '{ellip:+05.2f} +/- {ellip_err:04.2f}\t\t{theta:+05.2f} +/- {theta_err:04.2f}'

results_table = ''
for band in bands:
    print(band + ' | obs_id', 'amp [mK_CMB]', 'fwhm [arcmin]', 'ellipticity', 'theta [rad]', sep='\t\t')
    for wafer in wafers:
        print("\n".join([row_str.format(obs=obs_id, wafer=wafer, **results[obs_id][wafer][band])
                                for obs_id in sorted(results.keys())])+'\n')